<a href="https://colab.research.google.com/github/stravo1/Colab-Notebooks/blob/main/Langchain_OpenRouter_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv openai langchain langchain_openai langchainhub langchain_core langchain_community search-engine-parser nest_asyncio

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

In [3]:
from typing import Any, List, Mapping, Optional, Union, Sequence, Tuple, Dict, Iterator

from openai import OpenAI
from os import getenv

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.prompt_values import PromptValue
from langchain_core.messages.base import BaseMessage
from langchain_core.runnables.config import RunnableConfig

from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

from langchain import hub
from langchain.agents import AgentExecutor, create_json_chat_agent
from langchain_openai import ChatOpenAI

In [4]:
search = GoogleSearchAPIWrapper()


def top3_results(query):
    return search.results(query, 3)


search_tool = Tool(
    name="Google Search Snippets",
    description="Search Google for recent results.",
    func=top3_results,
)

tools = [search_tool]

In [5]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react-chat-json")

In [6]:
import json

llm = ChatOpenAI(
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=getenv("API_KEY"),
    model = "mistralai/mistral-7b-instruct:free"
)

# Construct the JSON agent
agent = create_json_chat_agent(llm, tools, prompt)

In [7]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [9]:
agent_executor.invoke({"input": "who won last IPL match?"})



> Entering new AgentExecutor chain...
```json
{
    "action": "Google Search Snippets",
    "action_input": "ipl match winner last match"
}
```[{'title': 'List of Indian Premier League seasons and results - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/List_of_Indian_Premier_League_seasons_and_results', 'snippet': 'The team that wins the final match is crowned the Indian Premier League champion. Altogether, thirteen teams have played in the past ten seasons of the IPL\xa0...'}, {'title': 'Yesterday IPL Match Result', 'link': 'https://www.sportskeeda.com/cricket/yesterday-ipl-match-result', 'snippet': 'Chennai Super Kings lifted their fifth IPL title as they defeated former champions, Gujarat Titans, in what turned out to be an epic finale at the Narendra\xa0...'}, {'title': '2023 Indian Premier League final - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/2023_Indian_Premier_League_final', 'snippet': 'Chennai Super Kings won the toss and elected to field. The match was origi

{'input': 'who won last IPL match?',
 'output': 'Chennai Super Kings won the 2023 Indian Premier League final.'}